# 利用PINNs求解泊松方程

本案例要求**MindSpore版本 >= 2.0.0**调用如下接口: *mindspore.jit，mindspore.jit_class，mindspore.jacrev*。

## 问题描述

本案例演示如何利用PINNs在不同几何体下求解一维、二维和三维泊松方程。

一维泊松方程定义为

$$
\Delta u = -\sin(4\pi x),
$$

二维泊松方程定义为

$$
\Delta u = -\sin(4\pi x)\sin(4\pi y),
$$

而三维方程定义为

$$
\Delta u = -\sin(4\pi x)\sin(4\pi y)\sin(4\pi z),
$$

很容易验证，以下函数分别满足二维和三维泊松方程

$$
u = \frac{1}{16\pi^2} \sin(4\pi x)\\
u = \frac{1}{32\pi^2} \sin(4\pi x)\sin(4\pi y), \\
u = \frac{1}{48\pi^2} \sin(4\pi x)\sin(4\pi y)\sin(4\pi z).
$$

如果在几何体边界按以上函数取狄利克雷边界条件，那么这些函数就是我们想要得到的解。因而，我们可以利用以上函数来验证结果。
对于一维问题，本案例使用一维数轴区间作为求解域，对于二维问题，本例演示在矩形，圆形，三角形和五边形区域求解方程，而对于三维问题，我们将在四面体，圆柱和圆锥区域内求解方程。

## 技术路径

MindFlow求解该问题的具体流程如下：

1. 创建训练数据集。
2. 构建模型。
3. 优化器。
4. 约束。
5. 模型训练。
6. 模型评估。

In [73]:
import time

from mindspore import context, nn, ops, jit
from mindflow import load_yaml_config

from src.model import create_model
from src.lr_scheduler import OneCycleLR
from src.dataset import create_dataset


context.set_context(mode=context.GRAPH_MODE, save_graphs=False, device_target="GPU")

# Load config
file_cfg = "poisson_cfg.yaml"
config = load_yaml_config(file_cfg)

## 创建数据集

本案例在求解域及边值条件进行随机采样，生成训练数据集与测试数据集。具体方法见``src/dataset.py``。设值``geom_name``来选择几何体，可选择rectangle, disk, triangle, pentagon, tetrahedon, cylinder和cone。

In [74]:
geom_name = "interval"
ds_train, n_dim = create_dataset(geom_name, config)

## 构建模型

本案例采用带3个隐藏层的多层感知器，并带有以下特点:

- 采用激活函数：$f(x) = x \exp(-x^2/(2e)) $

- 最后一层线性层使用weight normalization。

- 所有权重都采用``mindspore``的``HeUniform``初始化。

具体定义见``src/model.py``。

In [75]:
model = create_model(**config['model'][f'{n_dim}d'])

## 约束

在利用``mindflow``求解PDE时，我们需要写一个``mindflow.PDEWithLloss``的子类来定义控制方程，边界条件和损失函数。在求解区域内和边界上均采用L2损失，并利用``mindflow``的``MTLWeightedLoss``多目标损失函数将两个损失结合起来。

In [76]:
import sympy
from mindspore import numpy as ms_np
from mindflow import PDEWithLoss, MTLWeightedLoss, sympy_to_mindspore


class Poisson(PDEWithLoss):
    """Define the loss of the Poisson equation."""

    def __init__(self, model, n_dim):
        if n_dim == 1:
            var_str = "x,"
        elif n_dim == 2:
            var_str = "x y"
        elif n_dim == 3:
            var_str = "x y z"
        else:
            raise ValueError("`n_dim` can only be 2 or 3.")
        self.in_vars = sympy.symbols(var_str)
        self.out_vars = (sympy.Function("u")(*self.in_vars),)
        super(Poisson, self).__init__(model, self.in_vars, self.out_vars)
        self.bc_nodes = sympy_to_mindspore(self.bc(n_dim), self.in_vars, self.out_vars)
        self.loss_fn = MTLWeightedLoss(num_losses=2)

    def pde(self):
        """Define the gonvering equation."""
        poisson = 0
        src_term = 1
        sym_u = self.out_vars[0]
        for var in self.in_vars:
            poisson += sympy.diff(sym_u, (var, 2))
            src_term *= sympy.sin(4 * sympy.pi * var)
        poisson += src_term
        equations = {"poisson": poisson}
        return equations

    def bc(self, n_dim):
        """Define the boundary condition."""
        bc_term = 1
        for var in self.in_vars:
            bc_term *= sympy.sin(4 * sympy.pi * var)
        bc_term *= 1 / (16 * n_dim * sympy.pi * sympy.pi)
        bc_eq = self.out_vars[0] - bc_term
        equations = {"bc": bc_eq}
        return equations

    def get_loss(self, pde_data, bc_data):
        """Define the loss function."""
        res_pde = self.parse_node(self.pde_nodes, inputs=pde_data)
        res_bc = self.parse_node(self.bc_nodes, inputs=bc_data)
        loss_pde = ms_np.mean(ms_np.square(res_pde[0]))
        loss_bc = ms_np.mean(ms_np.square(res_bc[0]))
        return self.loss_fn((loss_pde, loss_bc))


# Create the problem
problem = Poisson(model, n_dim)

poisson: sin(4*pi*x) + Derivative(u(x), (x, 2))
    Item numbers of current derivative formula nodes: 2
bc: u(x) - sin(4*pi*x)/(16*pi**2)
    Item numbers of current derivative formula nodes: 2


## 优化器

本案例采用Adam优化器，并配合[Super-Convergence: Very Fast Training of Neural Networks Using Large Learning Rates](https://arxiv.org/abs/1708.07120)提出的动态学习率进行训练。动态学习率定义参见``src/lr_scheduler.py``。

In [77]:
n_epochs = 50

params = model.trainable_params() + problem.loss_fn.trainable_params()
steps_per_epoch = config['data']['domain']['size']//config['batch_size']
learning_rate = OneCycleLR(total_steps=steps_per_epoch*n_epochs, **config['optimizer'])
opt = nn.Adam(params, learning_rate=learning_rate)

## 模型训练

使用MindSpore>= 2.0.0的版本，可以使用函数式编程范式训练神经网络。

In [78]:
def train():
    # Create
    grad_fn = ops.value_and_grad(problem.get_loss, None, opt.parameters, has_aux=False)

    @jit
    def train_step(pde_data, bc_data):
        loss, grads = grad_fn(pde_data, bc_data)
        loss = ops.depend(loss, opt(grads))
        return loss

    def train_epoch(model, dataset, i_epoch):
        n_step = dataset.get_dataset_size()
        model.set_train()
        for i_step, (pde_data, bc_data) in enumerate(dataset):
            local_time_beg = time.time()
            loss = train_step(pde_data, bc_data)

            if i_step%50 == 0 or i_step + 1 == n_step:
                print("\repoch: {}, loss: {:>f}, time elapsed: {:.1f}ms [{}/{}]".format(
                    i_epoch, float(loss), (time.time() - local_time_beg)*1000, i_step + 1, n_step))

    for i_epoch in range(n_epochs):
        train_epoch(model, ds_train, i_epoch)

In [79]:
time_beg = time.time()
train()
print("End-to-End total time: {} s".format(time.time() - time_beg))

epoch: 0, loss: 1.603493, time elapsed: 8164.0ms [1/200]
epoch: 0, loss: 1.591017, time elapsed: 20.0ms [51/200]
epoch: 0, loss: 1.563642, time elapsed: 19.7ms [101/200]
epoch: 0, loss: 1.494738, time elapsed: 20.1ms [151/200]
epoch: 0, loss: 1.394417, time elapsed: 19.3ms [200/200]
epoch: 1, loss: 1.394938, time elapsed: 31.2ms [1/200]
epoch: 1, loss: 1.376357, time elapsed: 18.9ms [51/200]
epoch: 1, loss: 1.366150, time elapsed: 19.5ms [101/200]
epoch: 1, loss: 1.356578, time elapsed: 20.0ms [151/200]
epoch: 1, loss: 1.346048, time elapsed: 19.5ms [200/200]
epoch: 2, loss: 1.345783, time elapsed: 31.6ms [1/200]
epoch: 2, loss: 1.333922, time elapsed: 22.0ms [51/200]
epoch: 2, loss: 1.320294, time elapsed: 18.8ms [101/200]
epoch: 2, loss: 1.304355, time elapsed: 18.5ms [151/200]
epoch: 2, loss: 1.286624, time elapsed: 20.3ms [200/200]
epoch: 3, loss: 1.286230, time elapsed: 31.6ms [1/200]
epoch: 3, loss: 1.265824, time elapsed: 20.0ms [51/200]
epoch: 3, loss: 1.243438, time elapsed: 2

## 模型评估

可通过以下函数来计算模型的L2相对误差。

In [80]:
from src.utils import calculate_l2_error

n_samps = 5000  # Number of test samples
ds_test, _ = create_dataset(geom_name, config, n_samps)
calculate_l2_error(model, ds_test, n_dim)

Relative L2 error (domain): 0.0000
Relative L2 error (bc): 0.0000

